## HSコード別に輸出データの取得を行う

## 0.準備

In [33]:
import requests
import pandas as pd
from io import StringIO
import time
import gspread
from datetime import datetime
api_key = "cd892ddafd38c70af31204e5e2927a3573fa942e"
from google.colab import drive
import zipfile
import os
import re
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#statIDの管理
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

## 1.設定

In [34]:
syurui='HS'#品目別
start_year=2020 #2021
end_year=2022

cdArea→輸出国の指定、cdCat01→HSコード、cdCat02→金額・数量、cdCat03→税関

①月情報及び金額・数量の指定

In [35]:
#金額コードの指定
money_list=str(list(range(170, 501, 30))).replace('[','').replace(']','').replace(' ','')
#数量2コードの指定
quantity_list=str(list(range(160, 491, 30))).replace('[','').replace(']','').replace(' ','')
m_q_list=money_list+','+quantity_list

②StatIDの指定

In [36]:
worksheet = gc.open('貿易統計_対応表').get_worksheet(0)
# ワークシートからデータを取得してDataFrameに変換
data = worksheet.get_all_values()
trade_counter_df = pd.DataFrame(data[1:], columns=data[0])

In [37]:
trade_counter_df

,分類,year,statdataID
0,HS,2024,0004002162
1,HS,2023,0004002162
2,HS,2022,0003425301
3,HS,2021,0003425301
4,HS,2020,0003334000
...,...,...,...
67,OR,1992,0003258380
68,OR,1991,0003258380
69,OR,1990,0003258379
70,OR,1999,0003258379


In [38]:
def get_data(statdataID, startPosition, cat02,year):
    url = 'http://api.e-stat.go.jp/rest/3.0/app/getSimpleStatsData'
    params = {
        'appId': api_key,
        'lang': 'J',
        'statsDataId': statdataID,
        'metaGetFlg': 'N',
        'cntGetFlg': 'N',
        'explanationGetFlg': 'N',
        'annotationGetFlg': 'N',
        'sectionHeaderFlg': '1',
        'replaceSpChars': '0',
        'limit':'100000',
        'startPosition': str(startPosition),
        'cdCat02': cat02,
        #'cdArea':'50112',
        'cdTime':str(year)+'000000'

    }
    response = requests.get(url, params=params).text
    i_NEXT_KEY = response.find('"NEXT_KEY"')
    if i_NEXT_KEY >= 0:
        next_startPosition = response[response.find('"NEXT_KEY"'):].split('\n')[0].split(',')[-1][1:-1]  # NEXT_KEYの値部分
    else:
        # 10万件以下の場合はNEXT_KEYタグは存在しない
        next_startPosition = 1  # startPosition=1と同値
    df = pd.read_csv(StringIO(response[response.find('"VALUE"') + len('"VALUE"'):]))  # 実際の貿易統計のデータ部分
    return df, next_startPosition

## 2.データ取得実行

In [39]:
hs_counter_df = trade_counter_df[(trade_counter_df['分類'] == syurui) & (trade_counter_df['year'].astype(int) >= start_year)& (trade_counter_df['year'].astype(int) <= end_year)].reset_index(drop=True)

In [40]:
hs_counter_df

,分類,year,statdataID
0,HS,2022,0003425301
1,HS,2021,0003425301
2,HS,2020,0003334000


In [41]:
worksheet = gc.open('nation').get_worksheet(0)
data = worksheet.get_all_values()
nation_df = pd.DataFrame(data[1:], columns=data[0])

In [42]:
nation_df

,国,地域,地域(英語)
0,アメリカ合衆国,北アメリカ,North America
1,オーストラリア,オセアニア,Oceania
2,大韓民国,アジア,Asia
3,シンガポール,アジア,Asia
4,中華人民共和国,アジア,Asia
...,...,...,...
216,セウタ及びメリリア(西),アフリカ,Africa
217,セントヘレナ及びその附属諸島(英),アフリカ,Africa
218,グリーンランド(デンマーク),ヨーロッパ,Europe
219,フェロー諸島（デンマーク）,ヨーロッパ,Europe


In [ ]:
# @title データの取得

# セルの実行開始時刻を記録
start_time = datetime.now()
print('取得開始')


for l in range(len(hs_counter_df)): # start_year ~ end_yearのstatdataI
    all_dfs=[]
    statdataID=trade_counter_df['statdataID'][l]
    year=trade_counter_df['year'][l]
    cat02_list=m_q_list
    print('分類：',hs_counter_df['分類'][l],', 年度：',hs_counter_df['year'][l],', StatdataID：',hs_counter_df['statdataID'][l])


    syutoku_data=0
    startPosition = 1  # init
    # 10万件未満⇒next_startPosition = 1
    # 10万件以上⇒next_startPosition > 1
    df, next_startPosition = get_data(statdataID, startPosition,cat02_list,year)
    if df.columns[0]=='RESULT':
        syutoku_data+=len(df)
        print(' - - 取得完了:',syutoku_data,'行')
        break
    all_dfs.append(df)

    # 1回のAPI呼び出しあたり10万件未満(next_startPosition=1)になるまで取得し続ける.
    while int(next_startPosition) > 1:
          syutoku_data+=len(df)
          print(' - - 取得完了:',syutoku_data,'行')
          df, next_startPosition = get_data(statdataID, next_startPosition,cat02_list,year)
          all_dfs.append(df)
    syutoku_data+=len(df)
    print(' - - 取得完了:',syutoku_data,'行')


    final_df = pd.concat(all_dfs, ignore_index=True)
    del all_dfs
    # '国' 列のデータから '_' より右の国名だけを取得
    print(len(final_df))
    final_df['国'] = final_df['国'].str.split('_').str[1]
    # '統計品目表の数量・金額' 列のデータから '_' より右のフラグだけを取得
    final_df['数量・金額'] = final_df['統計品目表の数量・金額'].str.split('_').str[1]
    # '統計品目表の数量・金額' 列のデータから '_' より左の月だけを取得
    final_df['統計品目表の数量・金額'] = final_df['統計品目表の数量・金額'].str.split('_').str[0]
    selected_columns = ['時間軸(年次)','cat01_code', '統計品目表の数量・金額', '税関', '国','数量・金額','value']
    final_df_q = final_df[final_df['数量・金額'] == '数量2'][selected_columns]
    final_df_q.rename(columns={'value': '数量'}, inplace=True)


    selected_columns = ['時間軸(年次)','cat01_code', '統計品目表の数量・金額', '税関', '国','unit','数量・金額','value']
    final_df_m=final_df[final_df['数量・金額'] == '金額'][selected_columns]

    # カラムの名前を変更
    final_df_m.rename(columns={'value': '金額'}, inplace=True)
    del final_df
    # df1とdf2を指定の列で結合
    print(len(final_df_m),len(final_df_q))
    print(final_df_m.columns)
    print(final_df_q.columns)
    final_df = pd.merge(final_df_m, final_df_q, on=['cat01_code', '統計品目表の数量・金額', '税関', '国','時間軸(年次)'], how='inner')
    print(len(final_df),len(final_df_q))
    del final_df_q,final_df_m

    HS_master=pd.read_csv(f'/content/drive/MyDrive/政府統計/貿易統計/03_import/HScode/HSコードマスタ_{year}.csv')
    # final_df と HS_master を左結合
    merged_df = pd.merge(final_df, HS_master, left_on='cat01_code', right_on='HSコード', how='left')
    print(len(merged_df))
    merged_df=pd.merge(merged_df, nation_df, left_on='国', right_on='国', how='left')
    del final_df
    #merged_df=merged_df[['area_code','国','時間軸(年次)','cat02_code','統計品目表の数量・金額','cat03_code','税関','部数','部名', '類数', '類名', 'HSコード', '大項目', '中項目', '小項目', '細項目', '微細項目', '項目', '単位2','value']]
    merged_df=merged_df[['地域','国','時間軸(年次)','統計品目表の数量・金額','税関','部数','部名', '類数', '類名', 'HSコード', '大項目', '中項目', '小項目', '細項目', '微細項目', '項目', '金額','unit','数量','単位2']]
    column_mapping = {'時間軸(年次)':'年','統計品目表の数量・金額':'月','単位2': '数量単位','unit':'金額単位'}
    merged_df = merged_df.copy()  # データフレームをコピー
    merged_df.rename(columns=column_mapping, inplace=True)  # 変更を行う
    # データ取得日の日付を取得（例として今日の日付を使用）
    data_acquisition_date = datetime.now().strftime('%Y%m%d')
    # ファイル名に日付を組み込んでデータフレームをCSVファイルに書き出す
    #merged_df.to_csv(f'/content/drive/MyDrive/政府統計/貿易統計/04_output/HS item/数量/{year}_月次_税関別_{statdataID}_{data_acquisition_date}.csv', index=False)
    # ファイル名に日付を組み込んでデータフレームをCSVファイルに書き出す
    csv_filename = f'{year}_月次_税関別_{statdataID}_{data_acquisition_date}.csv'
    csv_path = f'/content/drive/MyDrive/政府統計/貿易統計/04_output/HS item/{csv_filename}'

    #類別の格納
    for rui in HS_master['部数'].unique():
        rui_merged_df = merged_df[merged_df['部数'] == rui]
        part_number = int(re.search(r'\d+', rui).group())  # 数字部分を抽出
        csv_filename = f'Export_{year}_{part_number}_port.csv'
        folder_path = f'/content/drive/MyDrive/政府統計/貿易統計/04_output/HS item/類別/{rui}/'
        csv_path = os.path.join(folder_path, csv_filename)

        # フォルダが存在しない場合は作成する
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        rui_merged_df.to_csv(csv_path, index=False)

    #国別の格納
    for nation in nation_df['地域'].unique():
        nation_merged_df = merged_df[merged_df['地域'] == nation]
        eng_nation=nation_df[nation_df['地域']==nation]['地域(英語)'].unique()[0]
        csv_filename = f'Export_{year}_{eng_nation}_port.csv'
        folder_path = f'/content/drive/MyDrive/政府統計/貿易統計/04_output/HS item/地域別/{nation}/'
        csv_path = os.path.join(folder_path, csv_filename)

        # フォルダが存在しない場合は作成する
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        nation_merged_df.to_csv(csv_path, index=False)
    #merged_df.to_csv(csv_path, index=False)

   #del merged_df

    # セルの実行終了時刻を記録
    end_time = datetime.now()
    # 経過時間を計算してプリント
    execution_time = end_time - start_time
    total_seconds = execution_time.total_seconds()
    minutes, seconds = divmod(int(total_seconds), 60)
    print(f"セルの実行時間: {minutes}分 {seconds}秒")
    print('')
print('取得終了')

取得開始
分類： HS , 年度： 2022 , StatdataID： 0003425301
 - - 取得完了: 100000 行
 - - 取得完了: 200000 行
 - - 取得完了: 300000 行
 - - 取得完了: 400000 行
 - - 取得完了: 500000 行
 - - 取得完了: 600000 行
 - - 取得完了: 700000 行
 - - 取得完了: 800000 行
 - - 取得完了: 900000 行
 - - 取得完了: 1000000 行
 - - 取得完了: 1100000 行
 - - 取得完了: 1200000 行
 - - 取得完了: 1300000 行
 - - 取得完了: 1400000 行
 - - 取得完了: 1500000 行
 - - 取得完了: 1600000 行
 - - 取得完了: 1700000 行
 - - 取得完了: 1800000 行
 - - 取得完了: 1900000 行
 - - 取得完了: 2000000 行
 - - 取得完了: 2100000 行
 - - 取得完了: 2200000 行
 - - 取得完了: 2300000 行
 - - 取得完了: 2400000 行
 - - 取得完了: 2500000 行
 - - 取得完了: 2600000 行
 - - 取得完了: 2700000 行
 - - 取得完了: 2800000 行
 - - 取得完了: 2900000 行
 - - 取得完了: 3000000 行
 - - 取得完了: 3100000 行
 - - 取得完了: 3200000 行
 - - 取得完了: 3300000 行
 - - 取得完了: 3400000 行
 - - 取得完了: 3500000 行
 - - 取得完了: 3600000 行
 - - 取得完了: 3700000 行
 - - 取得完了: 3800000 行
 - - 取得完了: 3900000 行
 - - 取得完了: 4000000 行
 - - 取得完了: 4100000 行
 - - 取得完了: 4200000 行
 - - 取得完了: 4300000 行
 - - 取得完了: 4400000 行
 - - 取得完了: 4500000 行
 - - 取得完了: 46000

In [44]:
df

,cat01_code,統計品目表(輸出),cat02_code,統計品目表の数量・金額,cat03_code,税関,area_code,国,time_code,時間軸(年次),unit,value
0,820231000,820231000,460,11月_数量2,50100,東京,50213,213_ドイツ,2024000000,2024年,NaN,0
1,820231000,820231000,460,11月_数量2,50100,東京,50304,304_アメリカ合衆国,2024000000,2024年,NaN,0
2,820231000,820231000,460,11月_数量2,50104,成田,50110,110_ベトナム,2024000000,2024年,NaN,0
3,820231000,820231000,460,11月_数量2,50104,成田,50111,111_タイ,2024000000,2024年,NaN,0
4,820231000,820231000,460,11月_数量2,50104,成田,50123,123_インド,2024000000,2024年,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,820810000,820810000,400,9月_数量2,50104,成田,50246,246_スロバキア,2024000000,2024年,NaN,0
99996,820810000,820810000,400,9月_数量2,50104,成田,50304,304_アメリカ合衆国,2024000000,2024年,NaN,0
99997,820810000,820810000,400,9月_数量2,50104,成田,50305,305_メキシコ,2024000000,2024年,NaN,0
99998,820810000,820810000,400,9月_数量2,50120,新潟,50110,110_ベトナム,2024000000,2024年,NaN,0


## 以下、旧コード

In [ ]:
# @title 重量データの取得

# セルの実行開始時刻を記録
start_time = datetime.now()
print('取得開始')
for i in range(len(hs_counter_df)):
  if i ==9999:
    continue
  else:
    startPosition = 0
    all_dfs = []
    cat02=quantity_list
    statdataID=trade_counter_df['statdataID'][i]
    year=trade_counter_df['year'][i]
    print('分類：',trade_counter_df['分類'][i],', 年度：',trade_counter_df['year'][i],', StatdataID：',trade_counter_df['statdataID'][i])
    for i in range(99999):  # 任意の大きな範囲を指定
        df = get_data(statdataID, startPosition, cat02,year)
        if len(df) >= 99999:
            all_dfs.append(df)
            startPosition += len(df) + 1
            print(' - - 取得完了:',startPosition-i-1,'行')
        if len(df) < 99999:
            all_dfs.append(df)
            startPosition += len(df)
            print(' - - 取得完了:',startPosition-i,'行')
            break  # 取得したデータが 100000 行未満なら終了

    final_df = pd.concat(all_dfs, ignore_index=True)
    # '国' 列のデータから '_' より右の国名だけを取得
    final_df['国'] = final_df['国'].str.split('_').str[1]
    # '統計品目表の数量・金額' 列のデータから '_' より左の月だけを取得
    final_df['統計品目表の数量・金額'] = final_df['統計品目表の数量・金額'].str.split('_').str[0]

    HS_master=pd.read_csv(f'/content/drive/MyDrive/政府統計/貿易統計/03_import/HScode/HSコードマスタ_{year}.csv')
    # final_df と HS_master を左結合
    merged_df = pd.merge(final_df, HS_master, left_on='cat01_code', right_on='HSコード', how='left')
    #merged_df=merged_df[['area_code','国','時間軸(年次)','cat02_code','統計品目表の数量・金額','cat03_code','税関','部数','部名', '類数', '類名', 'HSコード', '大項目', '中項目', '小項目', '細項目', '微細項目', '項目', '単位2','value']]
    merged_df=merged_df[['国','時間軸(年次)','統計品目表の数量・金額','税関','部数','部名', '類数', '類名', 'HSコード', '大項目', '中項目', '小項目', '細項目', '微細項目', '項目', '単位2','value']]
    column_mapping = {'時間軸(年次)':'年','統計品目表の数量・金額':'月','単位2': '数量単位','value': '数量',}
    merged_df = merged_df.copy()  # データフレームをコピー
    merged_df.rename(columns=column_mapping, inplace=True)  # 変更を行う
    # データ取得日の日付を取得（例として今日の日付を使用）
    data_acquisition_date = datetime.now().strftime('%Y%m%d')
    # ファイル名に日付を組み込んでデータフレームをCSVファイルに書き出す
    #merged_df.to_csv(f'/content/drive/MyDrive/政府統計/貿易統計/04_output/HS item/数量/{year}_月次_税関別_{statdataID}_{data_acquisition_date}.csv', index=False)
    # ファイル名に日付を組み込んでデータフレームをCSVファイルに書き出す
    csv_filename = f'{year}_月次_税関別_{statdataID}_{data_acquisition_date}.csv'
    csv_path = f'/content/drive/MyDrive/政府統計/貿易統計/04_output/HS item/数量/{csv_filename}'
    merged_df.to_csv(csv_path, index=False)

    # ZIPアーカイブに追加
    zip_filename = f'{year}_月次_税関別_{statdataID}_{data_acquisition_date}.zip'
    zip_path = f'/content/drive/MyDrive/政府統計/貿易統計/04_output/HS item/数量/{zip_filename}'

    with zipfile.ZipFile(zip_path, 'w') as zip_file:
        zip_file.write(csv_path, os.path.basename(csv_path))

    # 不要になったCSVファイルを削除
    os.remove(csv_path)

    # セルの実行終了時刻を記録
    end_time = datetime.now()
    # 経過時間を計算してプリント
    execution_time = end_time - start_time
    total_seconds = execution_time.total_seconds()
    minutes, seconds = divmod(int(total_seconds), 60)
    print(f"セルの実行時間: {minutes}分 {seconds}秒")
    print('')
print('取得終了')

取得開始
分類： HS , 年度： 2023 , StatdataID： 0004002162
 - - 取得完了: 99999 行
 - - 取得完了: 199999 行
 - - 取得完了: 299999 行
 - - 取得完了: 399999 行
 - - 取得完了: 499999 行
 - - 取得完了: 599999 行
 - - 取得完了: 699999 行
 - - 取得完了: 799999 行
 - - 取得完了: 899999 行
 - - 取得完了: 999999 行
 - - 取得完了: 1099999 行
 - - 取得完了: 1199999 行
 - - 取得完了: 1299999 行
 - - 取得完了: 1399999 行
 - - 取得完了: 1499999 行
 - - 取得完了: 1599999 行
 - - 取得完了: 1699999 行
 - - 取得完了: 1799999 行
 - - 取得完了: 1899999 行
 - - 取得完了: 1999999 行
 - - 取得完了: 2099999 行
 - - 取得完了: 2199999 行
 - - 取得完了: 2299999 行
 - - 取得完了: 2399999 行
 - - 取得完了: 2499999 行
 - - 取得完了: 2599999 行
 - - 取得完了: 2699999 行
 - - 取得完了: 2799999 行
 - - 取得完了: 2899999 行
 - - 取得完了: 2999999 行
 - - 取得完了: 3099999 行
 - - 取得完了: 3199999 行
 - - 取得完了: 3299999 行
 - - 取得完了: 3399999 行
 - - 取得完了: 3499999 行
 - - 取得完了: 3599999 行
 - - 取得完了: 3699999 行
 - - 取得完了: 3799999 行
 - - 取得完了: 3899999 行
 - - 取得完了: 3999999 行
 - - 取得完了: 4099999 行
 - - 取得完了: 4199999 行
 - - 取得完了: 4299999 行
 - - 取得完了: 4399999 行
 - - 取得完了: 4499999 行
 - - 取得完了: 4599999

In [ ]:
# @title 金額データの取得

# セルの実行開始時刻を記録
start_time = datetime.now()
# セルの実行開始時刻を記録
start_time = datetime.now()
print('取得開始')
for i in range(len(hs_counter_df)):
    startPosition = 0
    all_dfs = []
    cat02=money_list
    statdataID=trade_counter_df['statdataID'][i]
    year=trade_counter_df['year'][i]
    print('分類：',trade_counter_df['分類'][i],', 年度：',trade_counter_df['year'][i],', StatdataID：',trade_counter_df['statdataID'][i])
    for i in range(99999):  # 任意の大きな範囲を指定
        df = get_data(statdataID, startPosition, cat02)
        if len(df) >= 99999:
            all_dfs.append(df)
            startPosition += len(df) + 1
            print(' - - 取得完了:',startPosition-i-1,'行')
        if len(df) < 99999:
            all_dfs.append(df)
            startPosition += len(df)
            print(' - - 取得完了:',startPosition-i,'行')
            break  # 取得したデータが 100000 行未満なら終了

    final_df = pd.concat(all_dfs, ignore_index=True)
    # '国' 列のデータから '_' より右の国名だけを取得
    final_df['国'] = final_df['国'].str.split('_').str[1]
    # '統計品目表の数量・金額' 列のデータから '_' より左の月だけを取得
    final_df['統計品目表の数量・金額'] = final_df['統計品目表の数量・金額'].str.split('_').str[0]

    HS_master=pd.read_csv(f'/content/drive/MyDrive/政府統計/貿易統計/03_import/HScode/HSコードマスタ_{year}.csv')
    # final_df と HS_master を左結合
    merged_df = pd.merge(final_df, HS_master, left_on='cat01_code', right_on='HSコード', how='left')
    #merged_df=merged_df[['area_code','国','時間軸(年次)','cat02_code','統計品目表の数量・金額','cat03_code','税関','部数','部名', '類数', '類名', 'HSコード', '大項目', '中項目', '小項目', '細項目', '微細項目', '項目', 'unit','value']]
    merged_df=merged_df[['国','時間軸(年次)','統計品目表の数量・金額','税関','部数','部名', '類数', '類名', 'HSコード', '大項目', '中項目', '小項目', '細項目', '微細項目', '項目', 'unit','value']]
    column_mapping = {'時間軸(年次)':'年','統計品目表の数量・金額':'月','unit': '金額単位','value': '金額',}
    merged_df = merged_df.copy()  # データフレームをコピー
    merged_df.rename(columns=column_mapping, inplace=True)  # 変更を行う
    # データ取得日の日付を取得（例として今日の日付を使用）
    data_acquisition_date = datetime.now().strftime('%Y%m%d')
    # ファイル名に日付を組み込んでデータフレームをCSVファイルに書き出す
    merged_df.to_csv(f'/content/drive/MyDrive/政府統計/貿易統計/04_output/HS item/金額/{year}_月次_税関別_{statdataID}_{data_acquisition_date}.csv', index=False)

    # セルの実行終了時刻を記録
    end_time = datetime.now()
    # 経過時間を計算してプリント
    execution_time = end_time - start_time
    total_seconds = execution_time.total_seconds()
    minutes, seconds = divmod(int(total_seconds), 60)
    print(f"セルの実行時間: {minutes}分 {seconds}秒")

In [ ]:
merged_df

,国,年,月,税関,部数,部名,類数,類名,HSコード,大項目,中項目,小項目,細項目,微細項目,項目,金額単位,金額
0,中華人民共和国,2023年,1月,羽田,第1部,動物（生きているものに限る。）及び動物性生産品,第1類,動物（生きているものに限る。）,10619000.0,その他の動物（生きているものに限る。）,－ 哺乳類,－－ その他のもの,NaN,NaN,NaN,千円,0
1,中華人民共和国,2023年,1月,成田,第1部,動物（生きているものに限る。）及び動物性生産品,第1類,動物（生きているものに限る。）,10619000.0,その他の動物（生きているものに限る。）,－ 哺乳類,－－ その他のもの,NaN,NaN,NaN,千円,0
2,中華人民共和国,2023年,1月,関西空港,第1部,動物（生きているものに限る。）及び動物性生産品,第1類,動物（生きているものに限る。）,10619000.0,その他の動物（生きているものに限る。）,－ 哺乳類,－－ その他のもの,NaN,NaN,NaN,千円,0
3,中華人民共和国,2023年,2月,羽田,第1部,動物（生きているものに限る。）及び動物性生産品,第1類,動物（生きているものに限る。）,10619000.0,その他の動物（生きているものに限る。）,－ 哺乳類,－－ その他のもの,NaN,NaN,NaN,千円,0
4,中華人民共和国,2023年,2月,成田,第1部,動物（生きているものに限る。）及び動物性生産品,第1類,動物（生きているものに限る。）,10619000.0,その他の動物（生きているものに限る。）,－ 哺乳類,－－ その他のもの,NaN,NaN,NaN,千円,73388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437212,中華人民共和国,2023年,12月,石狩,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,千円,0
437213,中華人民共和国,2023年,12月,八戸,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,千円,0
437214,中華人民共和国,2023年,12月,秋田船川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,千円,0
437215,中華人民共和国,2023年,12月,沖縄地区,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,千円,0
